# Fetches air quality data from the OpenAQ API for the specified cities and stores it in a SQLite database.

In [3]:
#  Import necessary libraries and modules
import requests
import json
from store_data import create_connection, create_tables

In [4]:
def get_city_measurements(city):
    url = f"{OPENAQ_API_BASE}/measurements?city={city}&limit=100"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()["results"]
        return data
    else:
        print(f"Error fetching data for {city}: {response.status_code}")
        return []

In [5]:
def insert_measurement(conn, data):
    cursor = conn.cursor()

    cursor.execute("""
    INSERT INTO measurements (city, parameter, value, unit, date_utc, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, data)

    conn.commit()

In [6]:
OPENAQ_API_BASE = "https://api.openaq.org/v1"
CITIES = ["Stockholm", "Oslo", "Helsinki", "Copenhagen"]

In [7]:
def fetch_data():
    all_data = {}
    conn = create_connection()
    create_tables(conn)

    for city in CITIES:
        city_data = get_city_measurements(city)

        for entry in city_data:
            measurement_data = (
                entry["city"],
                entry["parameter"],
                entry["value"],
                entry["unit"],
                entry["date"]["utc"],
                entry["coordinates"]["latitude"],
                entry["coordinates"]["longitude"],
            )
            insert_measurement(conn, measurement_data)

        all_data[city] = city_data
        print(f"Fetched {len(city_data)} records for {city}")

    conn.close()

    with open("data.json", "w") as f:
        json.dump(all_data, f)

In [8]:
if __name__ == "__main__":
    fetch_data()

Fetched 177 records for Stockholm
Fetched 100 records for Oslo
Fetched 0 records for Helsinki
Fetched 0 records for Copenhagen
